In [16]:
# Imports
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from scikeras.wrappers import KerasRegressor
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import torch
import torch.nn as nn
from pytorch_forecasting.models import TemporalFusionTransformer






ModuleNotFoundError: No module named 'pytorch_forecasting'

In [ ]:
# Load SP500 and US macroeconomic data from CSV files
data = pd.read_csv('./data/SP500_data/GOOGL.csv', parse_dates=['Date'])
macro_data = pd.read_csv('./data/SP500_data/US_macroeconomics.csv', parse_dates=['date'])

data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)  # Set the date column as the index if not already set

# Merge datasets on the date field
merged_data = pd.merge(data, macro_data, left_on='Date', right_on='date', how='outer')

# Handle missing values: forward fill then backward fill to cover all gaps
merged_data.interpolate(method='ffill', inplace=True)
merged_data.interpolate(method='bfill', inplace=True)

# Drop the extra date column to avoid duplication
merged_data.drop(columns=['date'], inplace=True)

print(merged_data)



             Open         High          Low        Close    Adj Close  \
0       50.050049    52.082081    48.028027    50.220219    50.220219   
1       50.555557    54.594597    50.300301    54.209209    54.209209   
2       55.430431    56.796799    54.579578    54.754753    54.754753   
3       55.675674    55.855858    51.836838    52.487488    52.487488   
4       52.532532    54.054054    51.991993    53.053055    53.053055   
...           ...          ...          ...          ...          ...   
4787  2784.000000  2832.379883  2755.010010  2831.439941  2831.439941   
4788  2784.000000  2832.379883  2755.010010  2831.439941  2831.439941   
4789  2784.000000  2832.379883  2755.010010  2831.439941  2831.439941   
4790  2784.000000  2832.379883  2755.010010  2831.439941  2831.439941   
4791  2784.000000  2832.379883  2755.010010  2831.439941  2831.439941   

          Volume      CPI  Mortgage_rate  Unemp_rate        NASDAQ  \
0     44659096.0  189.800         5.7540         5.4 

In [ ]:
# Limit the dataset to the first 1000 rows
limited_data = merged_data.head(1000)

# Split the limited data into train and test sets
train_data = limited_data[:int(0.8*len(limited_data))]
test_data = limited_data[int(0.8*len(limited_data)):]

# Display the shapes of the train and test data
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)


Train data shape: (800, 13)
Test data shape: (200, 13)
           Open        High         Low       Close   Adj Close      Volume  \
0     50.050049   52.082081   48.028027   50.220219   50.220219  44659096.0   
1     50.555557   54.594597   50.300301   54.209209   54.209209  22834343.0   
2     55.430431   56.796799   54.579578   54.754753   54.754753  18256126.0   
3     55.675674   55.855858   51.836838   52.487488   52.487488  15247337.0   
4     52.532532   54.054054   51.991993   53.053055   53.053055   9188602.0   
..          ...         ...         ...         ...         ...         ...   
795  309.554565  313.273285  306.301300  308.308319  308.308319  12038549.0   
796  315.540527  317.317322  311.106110  317.057068  317.057068  12048939.0   
797  318.023010  321.006012  314.564575  320.130127  320.130127  24553822.0   
798  327.607605  329.574585  321.936951  322.677673  322.677673  31546422.0   
799  319.654663  327.827820  318.458466  325.700714  325.700714  13315471.0 

Epoch 1/100


ValueError: None values not supported.

In [ ]:

# Define the TFT model
class StockPriceTFT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.tft = TemporalFusionTransformer(
            hidden_size=config["hidden_size"],
            lstm_layers=config["lstm_layers"],
            dropout=config["dropout"],
            output_size=config["output_size"],
            loss=config["loss"],
            attention_head_size=config["attention_head_size"],
            max_encoder_length=config["max_encoder_length"],
            static_categoricals=config["static_categoricals"],
            static_reals=config["static_reals"],
            time_varying_categoricals_encoder=config["time_varying_categoricals_encoder"],
            time_varying_categoricals_decoder=config["time_varying_categoricals_decoder"],
            categorical_groups=config["categorical_groups"],
            time_varying_reals_encoder=config["time_varying_reals_encoder"],
            time_varying_reals_decoder=config["time_varying_reals_decoder"],
            x_reals=config["x_reals"],
            x_categoricals=config["x_categoricals"],
            hidden_continuous_size=config["hidden_continuous_size"],
            hidden_continuous_sizes=config["hidden_continuous_sizes"],
            embedding_sizes=config["embedding_sizes"],
            embedding_paddings=config["embedding_paddings"],
            embedding_labels=config["embedding_labels"],
            learning_rate=config["learning_rate"],
            log_interval=config["log_interval"],
            log_val_interval=config["log_val_interval"],
            log_gradient_flow=config["log_gradient_flow"],
            reduce_on_plateau_patience=config["reduce_on_plateau_patience"],
            monotone_constaints=config["monotone_constaints"],
            share_single_variable_networks=config["share_single_variable_networks"],
            causal_attention=config["causal_attention"],
            logging_metrics=config["logging_metrics"]
        )

    def forward(self, x):
        return self.tft(x)

# Configure the TFT model
config = {
    "hidden_size": 64,
    "lstm_layers": 2,
    "dropout": 0.1,
    "output_size": 1,
    "loss": nn.MSELoss(),
    "attention_head_size": 4,
    "max_encoder_length": 30,
    "static_categoricals": [],
    "static_reals": [],
    "time_varying_categoricals_encoder": [],
    "time_varying_categoricals_decoder": [],
    "categorical_groups": {},
    "time_varying_reals_encoder": ["price"],
    "time_varying_reals_decoder": ["price"],
    "x_reals": ["price"],
    "x_categoricals": [],
    "hidden_continuous_size": 16,
    "hidden_continuous_sizes": {},
    "embedding_sizes": {},
    "embedding_paddings": [],
    "embedding_labels": {},
    "learning_rate": 0.001,
    "log_interval": 10,
    "log_val_interval": 10,
    "log_gradient_flow": False,
    "reduce_on_plateau_patience": 10,
    "monotone_constaints": {},
    "share_single_variable_networks": False,
    "causal_attention": True,
    "logging_metrics": None
}

# Create the TFT model
model = StockPriceTFT(config)

# Prepare your data

# Train the model
optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
num_epochs = 100

for epoch in range(num_epochs):
    # Training loop
    model.train()
    for batch in train_data:
        optimizer.zero_grad()
        output = model(batch)
        loss = config["loss"](output, batch["target"])
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for batch in test_data:
            output = model(batch)
            val_loss += config["loss"](output, batch["target"]).item()
        val_loss /= len(test_data)
        print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}")

# Make predictions
model.eval()
with torch.no_grad():
    predictions = model(test_data)